In [ ]:
# Copy and pasted from https://github.com/rasmusbergpalm/nanograd/blob/main/nanograd.py

from typing import Union
from math import tanh


class Var:
    """
    A variable which holds a number and enables gradient computations.
    """

    def __init__(self, val: Union[float, int], parents=None):
        assert type(val) in {float, int}
        if parents is None:
            parents = []
        self.v = val
        self.parents = parents
        self.grad = 0.0

    def backprop(self, bp):
        self.grad += bp
        for parent, grad in self.parents:
            parent.backprop(grad * bp)

    def backward(self):
        self.backprop(1.0)

    def __add__(self: 'Var', other: 'Var') -> 'Var':
        return Var(self.v + other.v, [(self, 1.0), (other, 1.0)])

    def __mul__(self: 'Var', other: 'Var') -> 'Var':
        return Var(self.v * other.v, [(self, other.v), (other, self.v)])

    def __pow__(self, power: Union[float, int]) -> 'Var':
        assert type(power) in {float, int}, "power must be float or int"
        return Var(self.v ** power, [(self, power * self.v ** (power - 1))])

    def __neg__(self: 'Var') -> 'Var':
        return Var(-1.0) * self

    def __sub__(self: 'Var', other: 'Var') -> 'Var':
        return self + (-other)

    def __truediv__(self: 'Var', other: 'Var') -> 'Var':
        return self * other ** -1

    def tanh(self) -> 'Var':
        return Var(tanh(self.v), [(self, 1 - tanh(self.v) ** 2)])

    def relu(self) -> 'Var':
        return Var(self.v if self.v > 0.0 else 0.0, [(self, 1.0 if self.v > 0.0 else 0.0)])

    def __repr__(self):
        return "Var(v=%.4f, grad=%.4f)" % (self.v, self.grad)

In [ ]:
import math

In [ ]:
class adiff:
    
    def __init__(self, value, parents=None):
        try:
            assert type(value) in {float, int}, "Numbers please"
        except AssertionError as msg:
            print(msg)
        self.val = value
        self.gradient = 0.0
        self.parents = parents
        if parents == None:
            self.parents = []

    """The backprop, from Rasmusbergpalm's github"""
    # def backprop(self, bp):
    #     self.grad += bp
    #     for parent, grad in self.parents:
    #         parent.backprop(grad * bp)
    #     return self
    def backprop(self, value):
        self.gradient+= value
        for parent, gradient in self.parents:
            parent.backprop(gradient * value)
    
    def backward(self):
        self.backprop(1.0)

    def __add__(self, o_value):
        n_value = self.val + o_value.val
        return adiff(n_value, [(self, 1.0), (o_value, 1.0)])

    def __mul__(self, o_value):
        n_value = self.val * o_value.val
        return adiff(n_value, [(self, o_value.val), (o_value, self.val)])

    def __sub__(self, o_value):
        n_value = adiff(self.val - o_value.val, [(self, 1.0), (o_value, 1.0)])
        return n_value
        
    def __pow__(self, o_value):
        n_value = self.val**o_value.val
        return adiff(n_value, [(self, o_value.val*self.val**(o_value.val-1.0)), (o_value, n_value*math.log(self.val))])
    
    def __truediv__(self, o_value):
        n_value = self.val/o_value.val
        return adiff(n_value,[(self,1.0/o_value.val), (o_value,-self.val/o_value.val**2)])##!!!!!!

    #Returning a f-string, with the value and gradient
    def __repr__(self):
        return (f"adiff value = {self.val}, parents = {len(self.parents)}, gradient = {self.gradient}")

##divide,relu,sigmoid,softmax 


In [ ]:
a = adiff(5)
j = adiff(2)
g = a + j
q = a - j
p = a * j
#powah = g ** j
div = p / j
l = p * j
f = l + j*q
print("j,",j)
print("a,",a)
print("g add,",g)
print("q minus",q)
print("p mult",p)
#print("power,",powah)
print("Division,",div)
print("l = p * j",l)
print("f = l + 4*q",f)
print("\n\n backwrd")
for what,i in enumerate([g,q,p,div,l,f]):
    adiff.backward(i)
    print("\n",what,"\n")
    print("j,",j)
    print("a,",a)
    print("g add,",g)
    print("q minus",q)
    print("p mult",p)
    #print("power,",powah)
    print("Division,",div)
    print("l = p * j",l)
    print("f = l + 4*q",f)




j, adiff value = 2, parents = 0, gradient = 0.0
a, adiff value = 5, parents = 0, gradient = 0.0
g add, adiff value = 7, parents = 2, gradient = 0.0
q minus adiff value = 3, parents = 2, gradient = 0.0
p mult adiff value = 10, parents = 2, gradient = 0.0
Division, adiff value = 5.0, parents = 2, gradient = 0.0
l = p * j adiff value = 20, parents = 2, gradient = 0.0
f = l + 4*q adiff value = 26, parents = 2, gradient = 0.0


 backwrd

 0 

j, adiff value = 2, parents = 0, gradient = 1.0
a, adiff value = 5, parents = 0, gradient = 1.0
g add, adiff value = 7, parents = 2, gradient = 1.0
q minus adiff value = 3, parents = 2, gradient = 0.0
p mult adiff value = 10, parents = 2, gradient = 0.0
Division, adiff value = 5.0, parents = 2, gradient = 0.0
l = p * j adiff value = 20, parents = 2, gradient = 0.0
f = l + 4*q adiff value = 26, parents = 2, gradient = 0.0

 1 

j, adiff value = 2, parents = 0, gradient = 2.0
a, adiff value = 5, parents = 0, gradient = 2.0
g add, adiff value = 7, parents

In [ ]:
p.parents

[(adiff value = 5, parents = 0, gradient = 1.0, 2),
 (adiff value = 2, parents = 0, gradient = 1.0, 5)]

In [ ]:
###testing on Var
a = Var(5.0)
j = Var(2.0)
g = a + j
q = a - j
p = a * j
#powah = g ** j
div = p / j
l = p * j
f = l + j*q
print("j,",j)
print("a,",a)
print("g add,",g)
print("q minus",q)
print("p mult",p)
#print("power,",powah)
print("Division,",div)
print("l = p * j",l)
print("f = l + 4*q",f)
print("\n\n backwrd")
for what,i in enumerate([g,q,p,div,l,f]):
    Var.backward(i)
    print("\n",what,"\n")
    print("j,",j)
    print("a,",a)
    print("g add,",g)
    print("q minus",q)
    print("p mult",p)
    #print("power,",powah)
    print("Division,",div)
    print("l = p * j",l)
    print("f = l + 4*q",f)

j, Var(v=2.0000, grad=0.0000)
a, Var(v=5.0000, grad=0.0000)
g add, Var(v=7.0000, grad=0.0000)
q minus Var(v=3.0000, grad=0.0000)
p mult Var(v=10.0000, grad=0.0000)
Division, Var(v=5.0000, grad=0.0000)
l = p * j Var(v=20.0000, grad=0.0000)
f = l + 4*q Var(v=26.0000, grad=0.0000)


 backwrd

 0 

j, Var(v=2.0000, grad=1.0000)
a, Var(v=5.0000, grad=1.0000)
g add, Var(v=7.0000, grad=1.0000)
q minus Var(v=3.0000, grad=0.0000)
p mult Var(v=10.0000, grad=0.0000)
Division, Var(v=5.0000, grad=0.0000)
l = p * j Var(v=20.0000, grad=0.0000)
f = l + 4*q Var(v=26.0000, grad=0.0000)

 1 

j, Var(v=2.0000, grad=0.0000)
a, Var(v=5.0000, grad=2.0000)
g add, Var(v=7.0000, grad=1.0000)
q minus Var(v=3.0000, grad=1.0000)
p mult Var(v=10.0000, grad=0.0000)
Division, Var(v=5.0000, grad=0.0000)
l = p * j Var(v=20.0000, grad=0.0000)
f = l + 4*q Var(v=26.0000, grad=0.0000)

 2 

j, Var(v=2.0000, grad=5.0000)
a, Var(v=5.0000, grad=4.0000)
g add, Var(v=7.0000, grad=1.0000)
q minus Var(v=3.0000, grad=1.0000)
p mul

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=58866f15-1cff-4fef-9525-5c3070562370' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>